In [2]:
import mne
%pip install mne_bids
%pip install --upgrade mne_bids

from mne_bids import BIDSPath, print_dir_tree, write_raw_bids, make_dataset_description, update_sidecar_json

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
#Allows us to work with file paths
import os.path as op
#Makes file path handling simpler
from pathlib import Path

In [4]:
#Change the file path to your data's location
data_dir = Path(r"C:\N8_internship_code\source_data")
print_dir_tree(data_dir, max_depth=1)

|source_data\
|--- rest\
|--- video\


In [ ]:
children = [child for child in data_dir.iterdir()]
children

[WindowsPath('C:/N8_internship_code/source_data/rest'),
 WindowsPath('C:/N8_internship_code/source_data/video')]

In [6]:
#Change this to match the file number
dir_number = 0
files = [file for file in children[dir_number].iterdir()]

In [7]:
file_path = files[0]
file_path

WindowsPath('C:/N8_internship_code/source_data/rest/CLV002.set')

In [8]:
data = mne.io.read_raw_eeglab(file_path)

C:\Users\Ariana Williams\AppData\Local\Temp\ipykernel_17688\4240071171.py:1: RuntimeWarning: Data will be preloaded. preload=False or a string preload is not supported when the data is stored in the .set file
  data = mne.io.read_raw_eeglab(file_path)
C:\Users\Ariana Williams\AppData\Local\Temp\ipykernel_17688\4240071171.py:1: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  data = mne.io.read_raw_eeglab(file_path)
C:\Users\Ariana Williams\AppData\Local\Temp\ipykernel_17688\4240071171.py:1: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  data = mne.io.read_raw_eeglab(file_path)


In [9]:
data

<RawEEGLAB | CLV002.set, 65 x 481064 (481.1 s), ~238.6 MiB, data loaded>

In [10]:
#specify line frequency
data.info["line_freq"] = 50

In [11]:
bids_root = op.join(data_dir.parent, "bids_example")
bids_root

'C:\\N8_internship_code\\bids_example'

In [12]:
#Edit this information ot match your data
subject_id = "001"
task = "rest"

bids_path = BIDSPath(subject=subject_id, task=task, root=bids_root)
write_raw_bids(data, bids_path, overwrite=True, allow_preload=True, format="EEGLAB")

Writing 'C:\N8_internship_code\bids_example\participants.tsv'...
Writing 'C:\N8_internship_code\bids_example\participants.json'...
Writing 'C:/N8_internship_code/bids_example/sub-001/eeg/sub-001_space-CapTrak_electrodes.tsv'...
Writing 'C:/N8_internship_code/bids_example/sub-001/eeg/sub-001_space-CapTrak_coordsystem.json'...
The provided raw data contains annotations, but you did not pass an "event_id" mapping from annotation descriptions to event codes. We will generate arbitrary event codes. To specify custom event codes, please pass "event_id".
Used Annotations descriptions: ['S 20', 'S 21', 'boundary']
Writing 'C:\N8_internship_code\bids_example\sub-001\eeg\sub-001_task-rest_events.tsv'...
Writing 'C:\N8_internship_code\bids_example\sub-001\eeg\sub-001_task-rest_events.json'...
Writing 'C:\N8_internship_code\bids_example\dataset_description.json'...
Writing 'C:\N8_internship_code\bids_example\sub-001\eeg\sub-001_task-rest_eeg.json'...
Writing 'C:\N8_internship_code\bids_example\sub

C:\Users\Ariana Williams\AppData\Local\Temp\ipykernel_17688\1908755717.py:6: RuntimeWarning: Converting data files to EEGLAB format
  write_raw_bids(data, bids_path, overwrite=True, allow_preload=True, format="EEGLAB")


Writing 'C:\N8_internship_code\bids_example\sub-001\sub-001_scans.tsv'...
Wrote C:\N8_internship_code\bids_example\sub-001\sub-001_scans.tsv entry with eeg\sub-001_task-rest_eeg.set.


BIDSPath(
root: C:/N8_internship_code/bids_example
datatype: eeg
basename: sub-001_task-rest_eeg.set)

In [13]:
#creating a dataset description JSON file (separate to other data entry types) [for people who want a better description of their dataset than previously provided] 
# #{will overwrite any existing dataset_description.json file in the root of the BIDS directory}
make_dataset_description(
    path=bids_root,
    name="EEGManyLabs Resting State Study", 
    hed_version="1",
    dataset_type='raw',
    data_license="CCO",
    authors=["Ariana Williams", "Daniel Brady"],
    acknowledgements=None,
    how_to_acknowledge="Cite (Williams et al., 2025) when using this dataset",
    funding=["The NHS", "The Uk government"],
    ethics_approvals="Ethical approval was granted by the University of Leeds School of Psychology Ethics committee (12345 2025)",
    references_and_links="https://mne.tools/mne-bids/stable/whats_new_previous_releases.html",
    doi="doi:https://doi.org/10.1016/j.tins.2017.02.004",
    generated_by=[
        {
            "Name": "MNE-BIDS",
            "Version": "0.14",
            "Description": "Used to convert MEG data into BIDS format."
        },
        {
            "Name": "MNE-Python",
            "Version": "1.6.1",
            "Description": "Used for MEG preprocessing and analysis."
        }
    ],
    source_datasets=[
        {
            "URL": "https://example.com/source_dataset",
            "DOI": "10.1234/example.doi",
        }],
            overwrite=True,
            verbose=True)

Writing 'C:\N8_internship_code\bids_example\dataset_description.json'...


In [ ]:
#updating a specific parameter in a JSON [SIDECAR]
json_file = Path(r'C:\N8_internship_code\bids_example\sub-001\eeg\sub-001_task-rest_eeg.json').absolute()
# Create BIDSPath using the JSON file's directory as root
root = (r'C:\N8_internship_code\bids_example')

bids_path1 = BIDSPath(subject='001', task='rest',
                     suffix='eeg', extension='.json', datatype='eeg',
                     root=root)

entries = {'EEGReference' : "FCz",
           'SamplingFrequency': 1000.0,
           'PowerLineFrequency': 50.0,
           'SoftwareFilters': {
                "Anti-aliasing filter":{
                "half-amplitude cutoff (Hz)": 500,
                "Roll-off": "6dB/Octave"
                }
                },
            'TaskName': "Resting State",
            'TaskDescription': "Participants were asked to rest with their eyes closed for 5 minutes.",
            'Instructions': "Please close your eyes and relax. Do not think about anything in particular.",
            'CogAtlasID': "", 
            'CogPOID': "",    
            'Manufacturer': "Brain Products GmbH",
            'ManufacturersModelName':"Brain Products actiCHamp",
            'CapManufacturer': "EasyCap",
            'CapManufacturersModelName': "EasyCap 64-channel cap",
            'SoftwareVersions' : {
                'MNE-BIDS': "1.0.0",
                'MNE-Python': "1.6.1"
                },
            'DeviceSerialNumber': "123456789",
            'EEGChannelCount': 64,
            'ECGChannelCount': 2,
            'EMGChannelCount': 0,
            'EOGChannelCount': 0,
            'MiscChannelCount': 0,
            'TriggerChannelCount': 0,
            'RecordingDuration': 600,
            'RecordingType': "Continuous",
            'EpochLength': 2.0,
            'EEGGround':"Fpz", 
            'HeadCircumference': 58.0,
            'EEGPlacementScheme': "10 percent system",
            'HardwareFilters':{
               "ADC's decimation filter (hardware bandwidth limit)":{
                "-3dB cutoff point (Hz)": 480,
                "Filter order sinc response": 5
                }
                },
            'SubjectArtefactDescription': "No known artifacts were present during the recording.",
            'InstitutionName': "University of Leeds",
            'InstitutionAddress': "Leeds, UK", 
            'InstitutionalDepartmentName': "School of Psychology", 
            'ElectricalStimulation': "",
            'ElectricalStimulationParameters': ""}
           
update_sidecar_json(bids_path1, entries, verbose=True)

Writing 'C:\N8_internship_code\bids_example\sub-001\eeg\sub-001_task-rest_eeg.json'...


In [15]:
#updating a specific parameter in a JSON [COORDSYSTEM]
json_file = Path(r'C:\N8_internship_code\bids_example\sub-001\eeg\sub-001_space-CapTrak_coordsystem.json').absolute()
# Create BIDSPath using the JSON file's directory as root
root = Path(r'C:\N8_internship_code\bids_example')


bids_path1 = BIDSPath(subject='001', task=None,
                     suffix='coordsystem', extension='.json', datatype='eeg',
                     root=root, space='CapTrak')

entries = {'EEGCoordinateSystem':"CapTrak",
           'EEGCoordinateUnits':"m",
           'EEGCoordinateSystemDescription': "CapTrak is a 3D digitization system that provides accurate spatial coordinates for EEG electrode placement.",
           'FiducialsDescription': "Fiducials are anatomical landmarks used to align the EEG cap with the participant's head. The fiducials used in this study were Nasion, Left Preauricular, and Right Preauricular.",
           'FiducialsCoordinates': {
                'Nasion': [0.0, 0.0, 0.0],
                'Left Preauricular': [-0.1, 0.0, 0.0],
                'Right Preauricular': [0.1, 0.0, 0.0]
           },
           'FiducialsCoordinateSystem': "CapTrak",
            'FiducialsCoordinateUnits': "m",
            'FiducialsCoordinateSystemDescription': "CapTrak fiducials are defined in a 3D coordinate system that aligns with the participant's head.",
            'AnatomicalLandmarkCoordinates': {
                'Nasion': [0.0, 0.0, 0.0],
                'Left Preauricular': [-0.1, 0.0, 0.0],
                'Right Preauricular': [0.1, 0.0, 0.0]
            },
            'AnatomicalLandmarkCoordinateSystem': "CapTrak",
            'AnatomicalLandmarkCoordinateUnits': "m",
            'AnatomicalLandmarkCoordinateSystemDescription': "CapTrak anatomical landmarks are defined in a 3D coordinate system that aligns with the participant's head."
            }       
update_sidecar_json(bids_path1, entries, verbose=True)

Writing 'C:\N8_internship_code\bids_example\sub-001\eeg\sub-001_space-CapTrak_coordsystem.json'...


In [16]:
#updating a specific parameter in a JSON [EVENTS]
json_file = Path(r'C:\N8_internship_code\bids_example\sub-001\eeg\sub-001_task-rest_events.json').absolute()
# Create BIDSPath using the JSON file's directory as root
root = (r'C:\N8_internship_code\bids_example')


bids_path1 = BIDSPath(subject='001', task='rest',
                     suffix='events', extension='.json', datatype='eeg',
                     root=root)

entries = {'Onset':{'Description': "Onset time of the event in seconds relative to the start of the recording.", 'Units': "seconds"},
        'Duration':{'Description': "Duration of the event in seconds.", 'Units': "Seconds"},
        'TrialType':{'LongName':"Event Category", 
                     'Description': "Indicator of the type of action that is expected.", 
                     'Levels': {
                        'Start': "A red square appears on the screen to indicate the start of a trial.",
                        'Stop': "A green square appears on the screen to indicate the end of a trial"}
                     },
        'ResponseTime':{'Description': "Time taken by the participant to respond to the event in seconds.", 'Units': "seconds"},
        'HED': {'musc': "EMG-artifact"},
        'Stim File': {'Description': "File containing the stimulus presentation information for the event.", 'FileType': "CSV", 'FilePath': "sub-001_task-rest_stim.csv"},
        'Channel': {'Description': "Channel used to record the event.", 'Type': "Trigger"}
        }       
update_sidecar_json(bids_path1, entries, verbose=True)

Writing 'C:\N8_internship_code\bids_example\sub-001\eeg\sub-001_task-rest_events.json'...


In [17]:
#code for adding information to the participants.json file
#updating a specific parameter in a JSON
json_file = Path(r'C:\N8_internship_code\bids_example\participants.json').absolute()
# Create BIDSPath using the JSON file's directory as root
root = (r'C:\N8_internship_code\bids_example')


bids_path1 = BIDSPath(subject=None, task=None,
                     suffix='participants', extension='.json', datatype=None,
                     root=root)

entries = {'Partiipant ID':{'Description': "Unique Participant identifier"}, 
           'Species': {'Description': "Species of the participant"}, 
           'Age': {'Description': "Age of the participant at time of testing", 'Units': "Years"}, 
           'Sex': {'Description': "Biological sex of the participant", 
                   'Levels': {
                       "F":'Female',
                       "M":'Male'
                   }
                },
            'Handedness': {'Description': "Handedness of the participant",
                        'Levels': {
                            "R": "Right-handed",
                            "L": "Left-handed",
                            "A": "Ambidextrous"
                        }
                },
            'Strain': {'Description': "Strain of the participant, if applicable"},
            'Strain RRID': {'Description': "Strain RRI ID of the participant, if applicable"},
        }       
update_sidecar_json(bids_path1, entries, verbose=True)

Writing 'C:\N8_internship_code\bids_example\participants.json'...


This is the extended code, the first bit should be automatically done in the pipeline, I can edit this to just update some parameters

Necessary:
>
    1. Participant ID 

Recommended:
>
    2. Species
    3. Age
    set_meas_date()
    4. Sex
    5. Handedness
    6. Strain
    7. Strain RRID
>
Optional:
>
    - Additional participant information may be included to further bolster your metadata.

In [18]:
data.info.keys()

dict_keys(['acq_pars', 'acq_stim', 'ctf_head_t', 'description', 'dev_ctf_t', 'dig', 'experimenter', 'utc_offset', 'device_info', 'file_id', 'highpass', 'hpi_subsystem', 'kit_system_id', 'helium_info', 'line_freq', 'lowpass', 'meas_date', 'meas_id', 'proj_id', 'proj_name', 'subject_info', 'xplotter_layout', 'gantry_angle', 'bads', 'chs', 'comps', 'events', 'hpi_meas', 'hpi_results', 'projs', 'proc_history', 'custom_ref_applied', 'sfreq', 'dev_head_t', 'ch_names', 'nchan'])

In [19]:
from datetime import datetime, timezone
from dateutil.relativedelta import relativedelta

# Generates the (approx) birthdate of the participant based on the measurement date and age

data.set_meas_date(datetime(2015, 6, 7, tzinfo= timezone.utc))

recording_date = data.info["meas_date"]

Birthdate = recording_date - relativedelta(years=30)
print(Birthdate)


1985-06-07 00:00:00+00:00


In [20]:
data.info["subject_info"] = {
    "id": 1,
    "his_id": "sub-001",
    "last_name": "Doe",
    "first_name": "John",
    "middle_name": "A",
    "birthday": Birthdate,
    "sex": 2,
    "hand": 1,
    "weight": 70.0,
    "height": 175.0,
}

In [21]:
data.info["subject_info"]


<subject_info | id: 1, his_id: sub-001, last_name: Doe, first_name: John, middle_name: A, birthday: 1985-06-07 00:00:00+00:00, sex: 2, hand: 1, weight: 70.0, height: 175.0>

In [22]:
data.get_montage()

<DigMontage | 0 extras (headshape), 0 HPIs, 3 fiducials, 65 channels>

In [36]:
from datetime import datetime, timezone
from dateutil.relativedelta import relativedelta
from mne.transforms import Transform

data.info["device_info"] = {
    "type": _,
    "model": "_",
    "serial": "_",
    "site": "_"
    }



# Generates the (approx) birthdate of the participant based on the measurement date and age
data.set_meas_date(datetime(2015, 6, 7, tzinfo= timezone.utc))
recording_date = data.info["meas_date"]
Birthdate = recording_date - relativedelta(years=30)


data.info["subject_info"] = {
    "id": 1,
    "his_id": "sub-001",
    "last_name": "Doe",
    "first_name": "John",
    "middle_name": "A",
    "birthday": Birthdate,
    "sex": 2,
    "hand": 1,
    "weight": 70.0,
    "height": 175.0,
}

data.info["line_freq"] = 50

data.info["bads"] = ["C3", "F3" ]

data.info["description"] = "a restingstate dataset"

data.info["dev_head_t"] = Transform("meg", "head")

data.info["experimenter"] = "John Doe"

data.info["helium_info"] = {
    "he_level_raw": 20,
    "helium_level": 12,
    "orig_file_guid": "_",
    "meas_date": (datetime(2015, 6, 7, tzinfo= timezone.utc))
    }


In [45]:
data.info["proj_name"] = "EEGManyLabs restingstate"

RuntimeError: proj_name cannot be set directly.

In [37]:
data.info


<Info | 15 non-empty values
 bads: 2 items (C3, F3)
 ch_names: Fp1, Fz, F3, F7, FT9, FC5, FC1, C3, T7, TP9, CP5, CP1, Pz, P3, ...
 chs: 65 EEG
 custom_ref_applied: False
 description: a restingstate dataset
 device_info: 4 items (dict)
 dig: 68 items (3 Cardinal, 65 EEG)
 experimenter: John Doe
 helium_info: <helium_info | he_level_raw: 20.0, helium_level: 12.0, orig_file_guid: _, meas_date: 2015-06-07 00:00:00+00:00>
 highpass: 0.0 Hz
 line_freq: 50.0
 lowpass: 500.0 Hz
 meas_date: 2015-06-07 00:00:00 UTC
 nchan: 65
 projs: []
 sfreq: 1000.0 Hz
 subject_info: <subject_info | id: 1, his_id: sub-001, last_name: Doe, first_name: John, middle_name: A, birthday: 1985-06-07 00:00:00+00:00, sex: 2, hand: 1, weight: 70.0, height: 175.0>
>

In [25]:
import pandas as pd

In [26]:
participants_tsv = pd.read_csv(r'C:\N8_internship_code\bids_example\participants.tsv', sep='\t')

In [27]:
education = ["High School"]  

participants_tsv["Education"] = education
participants_tsv.to_csv(r'C:\N8_internship_code\bids_example\participants.tsv', sep= '\t', index=False, na_rep='n/a')

ValueError: Length of values (1) does not match length of index (2)

In [ ]:
#way to iterate through participant IDs and print them
ID_list = []
for participant_id in participants_tsv["participant_id"]:
    ID_list.append(participant_id)
    print(ID_list)

['sub-001']


In [ ]:
for _, row in participants_tsv.iterrows():
    participant_id = row["participant_id"]
    edu = row.get("education", "None")
    print(f"{participant_id}: Age = {edu}")


sub-001: Age = ['High School']


In [ ]:
import pandas as pd
participants_tsv = pd.read_csv(r'C:\N8_internship_code\bids_example\participants.tsv', sep='\t')
location = participants_tsv.loc[participants_tsv["participant_id"] == "sub-001"].index[0]
participants_tsv.loc[location, "Education"] = "None"
print (participants_tsv)

  participant_id  age  sex  hand  weight  height Education education
0        sub-001  NaN  NaN   NaN     NaN     NaN      None      none


In [ ]:
import pandas as pd
participants_tsv = pd.read_csv(r'C:\N8_internship_code\bids_example\participants.tsv', sep='\t')
participants_tsv.loc[participants_tsv["participant_id"] == "sub-001", "Education"] = "none"
print(participants_tsv)
participants_tsv.to_csv(r'C:\N8_internship_code\bids_example\participants.tsv', sep= '\t', index=False, na_rep='n/a')

  participant_id  age  sex  hand  weight  height Education
0        sub-001  NaN  NaN   NaN     NaN     NaN      none


In [ ]:
print(participants_tsv)


  participant_id  age  sex  hand  weight  height Education
0        sub-001  NaN  NaN   NaN     NaN     NaN      none


In [ ]:
participants_tsv.drop([0, 0])
print(participants_tsv)

  participant_id  age  sex  hand  weight  height Education
0        sub-001  NaN  NaN   NaN     NaN     NaN      none


In [ ]:
new_row = pd.DataFrame([{"participant_id": "sub-002", "age": 30, "sex":"M"}])
participants_tsv = pd.concat([participants_tsv, new_row], ignore_index=True)
print(participants_tsv)
participants_tsv.to_csv(r'C:\N8_internship_code\bids_example\participants.tsv', sep= '\t', index=False, na_rep='n/a')

  participant_id   age  sex  hand  weight  height Education
0        sub-001   NaN  NaN   NaN     NaN     NaN      none
1        sub-002  30.0    M   NaN     NaN     NaN       NaN
2        sub-002  30.0    M   NaN     NaN     NaN       NaN


In [ ]:
participants_tsv = participants_tsv.drop(index=1)
print(participants_tsv)
participants_tsv.to_csv(r'C:\N8_internship_code\bids_example\participants.tsv', sep= '\t', index=False, na_rep='n/a')

KeyError: '[1] not found in axis'

In [ ]:
import pandas as pd
participants_tsv = pd.read_csv(bids_root / "participants.tsv", sep="\t")
for i, participant_id in enumerate(participant_ids):
    kwargs = dict(
        species="homo sapiens",
        age=age[i],
        sex=sex[i],
        handedness=hand[i],
        lab=lab,
        replication=study,
        old_id=oldids[i],
        tod=tod[i],
        education_year=edu_y[i],
        education_degree=edu_deg[i],
    )
    participants_tsv = _update_tsv(
        participants_tsv, "sub-" + participant_id, **kwargs
    )
    #** split dictionary
    #keyword arguments
participants_tsv = participants_tsv.astype(
    {"age": "Int64", "education_year": "Int64"}
)
participants_tsv.education_degree = (
    participants_tsv.education_degree.str.title()
)
participants_tsv[
    [
        "participant_id",
        "species",
        "age",
        "sex",
        "handedness",
        "tod",
        "education_year",
        "education_degree",
        "replication",
        "lab",
        "old_id",
    ]
].to_csv(
    BIDS_ROOT / "participants.tsv", sep="\t", index=False, na_rep="n/a"
)




TypeError: unsupported operand type(s) for /: 'str' and 'str'

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
readme = op.join(bids_root, "README")
with open(readme, encoding="utf-8-sig") as fid:
    text = fid.read()
print(text)

References
----------
Appelhoff, S., Sanderson, M., Brooks, T., Vliet, M., Quentin, R., Holdgraf, C., Chaumon, M., Mikulan, E., Tavabi, K., Höchenberger, R., Welke, D., Brunner, C., Rockhill, A., Larson, E., Gramfort, A. and Jas, M. (2019). MNE-BIDS: Organizing electrophysiological data into the BIDS format and facilitating their analysis. Journal of Open Source Software 4: (1896).https://doi.org/10.21105/joss.01896

Pernet, C. R., Appelhoff, S., Gorgolewski, K. J., Flandin, G., Phillips, C., Delorme, A., Oostenveld, R. (2019). EEG-BIDS, an extension to the brain imaging data structure for electroencephalography. Scientific Data, 6, 103.https://doi.org/10.1038/s41597-019-0104-8




In [ ]:
Possible code:

#mne bids code
# Get the electrode coordinates
testing_data = mne.datasets.testing.data_path()
captrak_path = op.join(testing_data, "montage", "captrak_coords.bvct")
montage = mne.channels.read_dig_captrak(captrak_path)

# Rename the montage channel names only for this example, because as said
# before, coordinate and EEG data were not actually collected together
# Do *not* do this for your own data.
montage.rename_channels(dict(zip(montage.ch_names, raw.ch_names)))

# "attach" the electrode coordinates to the `raw` object
# Note that this only works for some channel types (EEG/sEEG/ECoG/DBS/fNIRS)
raw.set_montage(montage)

# show the electrode positions
raw.plot_sensors()


SyntaxError: invalid syntax (4037140334.py, line 1)

In [ ]:
possible code:

Let’s pause and check that the information that we’ve written out to the sidecar files that describe our data is correct.
# Get the sidecar ``.json`` file
sidecar_json_bids_path = bids_path.copy().update(suffix="meg", extension=".json")
sidecar_json_content = sidecar_json_bids_path.fpath.read_text(encoding="utf-8-sig")
print(sidecar_json_content)